In [1]:
import random
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from collections import defaultdict
from PIL import Image
from sklearn.metrics import ConfusionMatrixDisplay
from tensorflow import keras
from keras import layers

from keras.datasets import cifar10
import keras
from keras import layers
from PIL import Image, ImageFilter
from keras.datasets import mnist
import numpy as np
import glob
from sklearn.model_selection import train_test_split
from keras.utils import load_img, img_to_array
import matplotlib.pyplot as plt
from keras.layers import Input, Conv2D, Flatten, Dense, Conv2DTranspose, Reshape, Activation, LeakyReLU, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K
from keras.optimizers import Adam
import numpy as np
import cv2

2024-07-12 18:05:31.424415: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-12 18:05:31.446400: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-12 18:05:31.747916: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
RANDOM_SEED = 555
tf.random.set_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

In [3]:
IMAGE_SIZE = 256

def preprocess(array):
    """
    Normalizes the supplied array and reshapes it into the appropriate format.
    """

    #array = array.astype("float32") / 255.0
    array = np.reshape(array, (len(array), IMAGE_SIZE, IMAGE_SIZE, 3))
    return array

def one_depreprocess(array):
    """
    Normalizes the supplied array and reshapes it into the appropriate format.
    """

    array = array.astype("float32") * 255.0
    array = np.reshape(array, (IMAGE_SIZE, IMAGE_SIZE, 3))
    return array

def depreprocess(array):
    """
    Normalizes the supplied array and reshapes it into the appropriate format.
    """

    #array = array.astype("float32") * 255.0
    array = np.reshape(array, (len(array), IMAGE_SIZE, IMAGE_SIZE, 3))
    return array

In [4]:
train_data_path = '/home/takanolab/proglams_python/data' 
image_size = IMAGE_SIZE 
color_setting = 3 
folder = ['hatake','kawa','mori','tatemono'] 
class_number = len(folder)
print('Number of class : ', str(class_number))

Number of class :  4


In [5]:
X_image = []  
Y_label = []
for index, name in enumerate(folder):
  read_data = train_data_path + '/' + name
  files = glob.glob(read_data + '/*.png') #ここを変更。png形式のファイルを利用する場合のサンプルです。
  print('--- 読み込んだデータセットは', read_data, 'です。')
  num=0
  for i, file in enumerate(files):
    if color_setting == 1:
      img = load_img(file, color_mode = 'grayscale' ,target_size=(image_size, image_size))  
    elif color_setting == 3:
      img = load_img(file, color_mode = 'rgb' ,target_size=(image_size, image_size))
    array = img_to_array(img)
    X_image.append(array)
    num +=1
    Y_label.append(index)
  print('index: ',index,' num:',num)

X_image = np.array(X_image)
Y_label = np.array(Y_label)

--- 読み込んだデータセットは /home/takanolab/proglams_python/data/hatake です。
index:  0  num: 144
--- 読み込んだデータセットは /home/takanolab/proglams_python/data/kawa です。
index:  1  num: 144
--- 読み込んだデータセットは /home/takanolab/proglams_python/data/mori です。
index:  2  num: 144
--- 読み込んだデータセットは /home/takanolab/proglams_python/data/tatemono です。
index:  3  num: 144


In [6]:
train_images, valid_images ,train_labels ,valid_labels = train_test_split(X_image,Y_label,test_size=0.20,shuffle = True)
x_train = train_images
y_train = train_labels
x_test = valid_images
y_test = valid_labels

Z_DIM=100

In [7]:
def r_loss(y_true, y_pred):
  return K.mean(K.square(K.log(y_true + 1) - K.log(y_pred + 1)), axis=-1)

Encoder_model = keras.models.load_model("/home/takanolab/davbi/development/Public-2021065/ex1/newmodel/ex1_model1/", custom_objects={"r_loss": r_loss })
Encoder_model.summary()

2024-07-12 18:07:13.078307: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-12 18:07:24.815670: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-12 18:07:24.816152: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 256, 256, 3)]     0         
                                                                 
 encoder_conv_0 (Conv2D)     (None, 256, 256, 3)       84        
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 256, 256, 3)       0         
                                                                 
 encoder_conv_1 (Conv2D)     (None, 128, 128, 16)      448       
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 128, 128, 16)      0         
                                                                 
 encoder_conv_2 (Conv2D)     (None, 64, 64, 32)        4640      
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 64, 64, 32)        0   

In [8]:
layer_name = 'encoder_output'
intermediate_layer_model = Model(inputs=Encoder_model.input,
                                 outputs=Encoder_model.get_layer(layer_name).output)

In [9]:
intermediate_layer_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 256, 256, 3)]     0         
                                                                 
 encoder_conv_0 (Conv2D)     (None, 256, 256, 3)       84        
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 256, 256, 3)       0         
                                                                 
 encoder_conv_1 (Conv2D)     (None, 128, 128, 16)      448       
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 128, 128, 16)      0         
                                                                 
 encoder_conv_2 (Conv2D)     (None, 64, 64, 32)        4640      
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 64, 64, 32)        0     

In [ ]:
intermediate_layer_model.save